# Flowcharts for MCCE_Benchmarking:

Note: GitHub markdown apparently cannot render the mermaid graphs despite saying it does.

---
---

# 2-14-2024
---
# Refactoring MCCE benchmark cli ([Issue 16](https://github.com/GunnerLab/MCCE_Benchmarking/issues/16))

## Purpose:
To allow benchmarking viz the curated, experimental pKa database (v1) or viz MCCE runs (two runs with completed step4).

## Completion status:
The refactoring is not complete yet as it entails creating two different tracks (or entry points), along with automated tests.
* 'experimental_pkas': 80%
* 'mcce_runs': 30%

## What the command line commands would be (w/o renaming main EP):
### Entry point 'experimental_pkas':
 1. Setup the pdbs folder and files for the user. NO GO: `benchmarks_dir` already exists
 ```
 # No input means `benchmarks_dir` = ./mcce_benchmarks:

 >mccebench experimental_pkas benchmark_setup


 # With input:

 >mccebench experimental_pkas benchmark_setup -benchmarks_dir <different name>


 ```
 2. Analysis (default for `pct_complete` not yet determined):
 ```
 >mccebench experimental_pkas analyze -pct_complete <float> [-benchmarks_dir: optional if default was used in `benchmark_setup`]
 ```

### Entry point 'mcce_runs':
NOTE: I think there is no need for the -eps parameter since we would want to compare, say e8 vs e4 (default); Updated the flowchart accordingly.  
NO GO: User selected -titr_type = 'eh' with -reference_dir = 'parse.e4': this reference set is for ph titrations.
```
 >mccebench mcce_runs -new_calc_dir <new>, -reference_dir <ref> -titr_type <ph or eh>


```

---
## Flowchart for the two entry points:

```mermaid
graph TD
    exp[Entry point: 'experimental_pkas'
    default script: dry prot, 4 steps w/mfe] -->|Input: 'benchmards_dir'| dir{dir exists?}
    dir -->|N| ng1[STOP:
    Rename existing dir or
    Change 'benchmarks_dir']
    dir -->|Y| sub1{Subcommand choice:
    1. 'benchmark_setup'
    2. 'analyze'}
    sub1 -->| 'benchmark_setup' |do1[Actions performed:
    1. Data setup
    2. Scheduling setup
    3. Launch]
    sub1 -->| 'analyze' | prob1[Problem:
    Are there enough completed runs?
    Implement with a 'percentage' user input?
    => Function needed as initial check
    to obtain the completed entries in
    the book file & launch the analysis
    if % is met.]
    prob1 --> runs1{Enough completed runs?}
    runs1 -->| Y | rpt1[Final outputs:
    * all_pkas file
    * Matched pKas file
    * Residue stats
    * Conformers throughput per step\n using runtimes & conformer counts
    * Plots
    * Anything else?]
    runs1 --> | N | msg1(Try '>experimental_pkas analyze' later)

    mc[Entry point: 'mcce_runs'] -->|Inputs:
    2 completed runs:
    'new_calc_dir', 'reference_dir';
    'titr_type'| ref{Which 'reference_dir'?}
    ref -->|ref dir is e.g. 'parse.e4' from pKaDB
    Applicable only to pH titrations| comp2[Use 'all_pkas.e4' file for comparison]
    comp2 --> mcpka[Analysis outputs:
    * Matched, then diffed pKa values
    * Plot new vs ref for all numeric fields in pK.out]
    ref -->|ref dir is another mcce output dir
    ASSUMED: runs of same prot| rptmc[Analysis outputs:
    * Diffed pK.out
    * Residue stats
    * Plot new vs ref for all numeric fields in pK.out
    * Anything else?]
    mcpka -.-> note[Problem:
    Analysis will depend on the
    contents of 'parse.e4' dir:
    full output, partial
    or just 'all_pkas.e4' file?]
    rptmc -.-> note
```